Step0-- analyze and store info about the camera pixel-by-pixel DC model, saturation point)
Step1-- import raw data from h5 files, clip the saturated pixels, subtract the DC noise, save data as additional npy arrays
Step2-- *** Use the data to generate camera response functions and convert the multiple exposure times into a radiance map.**

In [1]:

import os
import numpy as np
import logging

 # process_and_save.py

from Step2_image_radiance_dev import load_clipped_denoised_data, process_hdr_images, save_crf_data

# Your existing setup code here
directory = '/Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed/'
experiment_title = 'Water_immersed'
base_data_folder = '240817_data_95xSmax'

# Add a parameter for the number of image sets to process
num_sets_to_process = 5  # Change this value to process more or fewer sets

# Load the processed data
loaded_data = load_clipped_denoised_data(directory, experiment_title, base_data_folder)

# Process HDR images and get all necessary data
processed_data = process_hdr_images(directory, experiment_title, base_data_folder, smoothing_lambda=1000, num_sets=num_sets_to_process)

# Save CRF data for future analysis
crf_file = save_crf_data(processed_data, directory, experiment_title, base_data_folder)

print(f"CRF data saved to: {crf_file}")


Processing Water_immersed_808_BP1300
Images shape: (15, 640, 512)
Exposure times shape: (15,)
Slinear shape: (640, 512)
Sd shape: (640, 512)
bias shape: (640, 512)
Zmax_precomputed shape: (15, 640, 512)
Processing Water_immersed_670_BP1200
Images shape: (15, 640, 512)
Exposure times shape: (15,)
Slinear shape: (640, 512)
Sd shape: (640, 512)
bias shape: (640, 512)
Zmax_precomputed shape: (15, 640, 512)
Processing Water_immersed_760_BP1350
Images shape: (15, 640, 512)
Exposure times shape: (15,)
Slinear shape: (640, 512)
Sd shape: (640, 512)
bias shape: (640, 512)
Zmax_precomputed shape: (15, 640, 512)
Processing Water_immersed_760_BP1250
Images shape: (15, 640, 512)
Exposure times shape: (15,)
Slinear shape: (640, 512)
Sd shape: (640, 512)
bias shape: (640, 512)
Zmax_precomputed shape: (15, 640, 512)
Processing Water_immersed_670_BP1300
Images shape: (15, 640, 512)
Exposure times shape: (15,)
Slinear shape: (640, 512)
Sd shape: (640, 512)
bias shape: (640, 512)
Zmax_precomputed shape: 

In [2]:
# generate_report.py

import os
import numpy as np
import logging
from Step2_image_radiance_dev import load_crf_data, generate_multi_page_report, adaptive_weight

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Your existing setup code here
directory = '/Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed/'
experiment_title = 'Water_immersed'

# Load the saved CRF data
crf_file = os.path.join(directory, "data", "final_data", f"{experiment_title}_crf_data.npz")
processed_data = load_crf_data(crf_file)

logger.info("Loaded processed data:")
for i, data in enumerate(processed_data):
    logger.info(f"\nData item {i+1}:")
    for key, value in data.items():
        if isinstance(value, np.ndarray):
            logger.info(f"  {key} shape: {value.shape}")
            if np.issubdtype(value.dtype, np.number):  # Check if the array contains numeric data
                if np.isnan(value).any() or np.isinf(value).any():
                    logger.warning(f"  Warning: {key} contains NaN or inf values")
                logger.info(f"  {key} min: {np.min(value)}, max: {np.max(value)}")
            else:
                logger.info(f"  {key} contains non-numeric data")
        else:
            logger.info(f"  {key}: {value}")

# Generate the multi-page report
if processed_data:
    try:
        generate_multi_page_report(processed_data, directory, experiment_title, adaptive_weight)
        logger.info("Report generation complete")
    except Exception as e:
        logger.error(f"Error generating report: {str(e)}")
        import traceback
        logger.error(traceback.format_exc())
else:
    logger.warning("No data to generate report. Please check the CRF data file.")

logger.info("Script execution complete")

INFO:__main__:Loaded processed data:
INFO:__main__:
Data item 1:
INFO:__main__:  key shape: ()
INFO:__main__:  key contains non-numeric data
INFO:__main__:  radiance_map shape: (640, 512)
INFO:__main__:  radiance_map min: 3.3872790336608887, max: 11.958382606506348
INFO:__main__:  response_curve shape: (4421,)
INFO:__main__:  response_curve min: 3.264330194319991, max: 9.652338163564963
INFO:__main__:  z_min shape: ()
INFO:__main__:  z_min min: 0, max: 0
INFO:__main__:  z_max shape: ()
INFO:__main__:  z_max min: 4420.19406599884, max: 4420.19406599884
INFO:__main__:  intensity_samples shape: (20194, 15)
INFO:__main__:  intensity_samples min: 0.0, max: 3843.335693359375
INFO:__main__:  log_exposures shape: (20194, 15)
INFO:__main__:  log_exposures min: -4.9600429843612215, max: 12.272132385503335
INFO:__main__:
Data item 2:
INFO:__main__:  key shape: ()
INFO:__main__:  key contains non-numeric data
INFO:__main__:  radiance_map shape: (640, 512)
INFO:__main__:  radiance_map min: 3.812320

Loading CRF data from: /Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed/data/final_data/Water_immersed_crf_data.npz
Keys in the loaded file:
  key_0
  radiance_map_0
  response_curve_0
  z_min_0
  z_max_0
  intensity_samples_0
  log_exposures_0
  key_1
  radiance_map_1
  response_curve_1
  z_min_1
  z_max_1
  intensity_samples_1
  log_exposures_1
  key_2
  radiance_map_2
  response_curve_2
  z_min_2
  z_max_2
  intensity_samples_2
  log_exposures_2
  key_3
  radiance_map_3
  response_curve_3
  z_min_3
  z_max_3
  intensity_samples_3
  log_exposures_3
  key_4
  radiance_map_4
  response_curve_4
  z_min_4
  z_max_4
  intensity_samples_4
  log_exposures_4
  key_5
  radiance_map_5
  response_curve_5
  z_min_5
  z_max_5
  intensity_samples_5
  log_exposures_5
  key_6
  radiance_map_6
  response_curve_6
  z_min_6
  z_max_6
  intensity_samples_6
  log_exposures_6
  key_7
  radiance_m

INFO:__main__:  log_exposures min: -5.232906068258411, max: 13.110373522494564
INFO:__main__:
Data item 7:
INFO:__main__:  key shape: ()
INFO:__main__:  key contains non-numeric data
INFO:__main__:  radiance_map shape: (640, 512)
INFO:__main__:  radiance_map min: 3.4240715503692627, max: 11.20108699798584
INFO:__main__:  response_curve shape: (4421,)
INFO:__main__:  response_curve min: 3.1734786388610376, max: 10.131245271179322
INFO:__main__:  z_min shape: ()
INFO:__main__:  z_min min: 0, max: 0
INFO:__main__:  z_max shape: ()
INFO:__main__:  z_max min: 4420.19406599884, max: 4420.19406599884
INFO:__main__:  intensity_samples shape: (17309, 15)
INFO:__main__:  intensity_samples min: 0.0, max: 3718.32763671875
INFO:__main__:  log_exposures shape: (17309, 15)
INFO:__main__:  log_exposures min: -5.209532928832147, max: 11.493863144619137
INFO:__main__:
Data item 8:
INFO:__main__:  key shape: ()
INFO:__main__:  key contains non-numeric data
INFO:__main__:  radiance_map shape: (640, 512)
I

Error in plot_weighting_function: x and y must have same first dimension, but have shapes (4422,) and (1, 4422)
Debug: intensity_samples shape: (20194, 15)
Debug: log_exposures shape: (20194, 15)
Debug: response_curve shape: (4421,)
Debug: z_min: 0, z_max: 4420.19406599884
plot_log_log_crf - pixel_values shape: (4421,), response_curve shape: (4421,)


INFO:Step2_image_radiance_dev:Added report page for Water_immersed_808_BP1300


...

INFO:Step2_image_radiance_dev:Added report page for Water_immersed_808_BP1350


Error in plot_weighting_function: x and y must have same first dimension, but have shapes (4422,) and (1, 4422)
Debug: intensity_samples shape: (21010, 15)
Debug: log_exposures shape: (21010, 15)
Debug: response_curve shape: (4421,)
Debug: z_min: 0, z_max: 4420.19406599884
plot_log_log_crf - pixel_values shape: (4421,), response_curve shape: (4421,)


INFO:Step2_image_radiance_dev:Added report page for Water_immersed_670_BP1250


Error in plot_weighting_function: x and y must have same first dimension, but have shapes (4422,) and (1, 4422)
Debug: intensity_samples shape: (21322, 15)
Debug: log_exposures shape: (21322, 15)
Debug: response_curve shape: (4421,)
Debug: z_min: 0, z_max: 4420.19406599884
plot_log_log_crf - pixel_values shape: (4421,), response_curve shape: (4421,)


INFO:Step2_image_radiance_dev:Added report page for Water_immersed_760_BP1300
INFO:Step2_image_radiance_dev:Finished processing all files.
INFO:Step2_image_radiance_dev:Report saved as /Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed/data/hdr_report_Water_immersed_20240817_173828.pdf
INFO:__main__:Report generation complete
INFO:__main__:Script execution complete


In [3]:
# #debugging cell -- print out info about the data stored in the npz file


# import numpy as np
# import os

# def inspect_npz_file(file_path):
#     print(f"Inspecting file: {file_path}")
    
#     with np.load(file_path, allow_pickle=True) as data:
#         print("Keys in the file:")
#         for key in data.keys():
#             value = data[key]
#             if isinstance(value, np.ndarray):
#                 print(f"  {key}: ndarray with shape {value.shape}, dtype: {value.dtype}")
#                 if value.size > 0:
#                     print(f"    First few elements: {value.flatten()[:5]}")
#             else:
#                 print(f"  {key}: {type(value)}")
#                 print(f"    Value: {value}")

# # Update this path to match your file location
# npz_file_path = "/Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed/data/final_data/Water_immersed_crf_data.npz"

# inspect_npz_file(npz_file_path)

Inspecting file: /Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed/data/final_data/Water_immersed_crf_data.npz
Keys in the file:
  key_0: ndarray with shape (), dtype: <U25
    First few elements: ['Water_immersed_808_BP1300']
  radiance_map_0: ndarray with shape (640, 512), dtype: float32
    First few elements: [4.8048906 4.929789  5.018448  4.7511334 5.0964937]
  response_curve_0: ndarray with shape (4421,), dtype: float64
    First few elements: [3.26433019 3.26714737 3.27117292 3.27636657 3.28268805]
  z_min_0: ndarray with shape (), dtype: int64
    First few elements: [0]
  z_max_0: ndarray with shape (), dtype: float64
    First few elements: [4420.194066]
  intensity_samples_0: ndarray with shape (20194, 15), dtype: float32
    First few elements: [0. 0. 0. 0. 0.]
  log_exposures_0: ndarray with shape (20194, 15), dtype: float64
    First few elements: [-3.56896034 -2.